# Smart signatures
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [24]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [25]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [26]:
from pyteal import *

In [27]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [28]:
from pyteal import *

In [29]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


#### Check Purestake API

In [30]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19804036


## The Cash Mashine
A slightly more complicated contract: require a "password" in the transaction note

#### Step 1: The programmer writes down the conditions as a PyTeal program
* Depending on `Txn.note()` of the *proposed transaction*, this can be **True** or **False**

In [34]:
import random
from random import randrange
a = Int( randrange(2**32-1) )   

cashmachine_pyteal = And(
    Txn.note() == Bytes('{"4711"}'),
    a == a
)

# Security missing ... do not copy-paste

#### Step 2: Compile PyTeal -> Teal
* No need to print and inspect the TEAL program

In [35]:
cashmachine_teal = compileTeal(cashmachine_pyteal, Mode.Signature, version=3)
print(cashmachine_teal)

#pragma version 3
txn Note
byte "{\"4711\"}"
==
int 2077524607
int 2077524607
==
&&


#### Step 3: Compile Teal -> Bytecode for AVM
* `hash` contains the address $\longleftarrow$ corresponds to the `[public]`
* `result` contains the compiled code $\longleftarrow$ corresponds to the `[private]`
* Nothing yet to see at the Hash/Public address

In [37]:
# compile Teal -> Bytecode
Cashmachine = algod_client.compile(cashmachine_teal)
Cashmachine

{'hash': 'GWCINQHW75IVFL53I65WUENTFY6XDS5NVN35UC4EXWEFXWRUY55NMZ2N74',
 'result': 'AyAB/4TS3gcmAQh7IjQ3MTEifTEFKBIiIhIQ'}

#### Step 4: Alice funds and deploys the smart signature

In [38]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.5*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Cashmachine['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19804121.
Waiting for round 19804121 to finish.
Waiting for round 19804122 to finish.
Transaction V2VTCNGA7FPNFVTTJMWZLPAZWJGU2D2ND6RH7XXZW5GMV5UYPP3Q confirmed in round 19804123.


In [39]:
# Look on Algoexplorer.io at the smart signature. 
# The smart signature is funded.
print('http://testnet.algoexplorer.io/address/'+Cashmachine['hash'])

http://testnet.algoexplorer.io/address/GWCINQHW75IVFL53I65WUENTFY6XDS5NVN35UC4EXWEFXWRUY55NMZ2N74


#### Step 5: Alice informs Bob

In [41]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Cashmachine['result'])
print("Address of smart signature: ", Cashmachine['hash'])

Alice communicates to Bob the following
Compiled smart signature: AyAB/4TS3gcmAQh7IjQ3MTEifTEFKBIiIhIQ
Address of smart signature:  GWCINQHW75IVFL53I65WUENTFY6XDS5NVN35UC4EXWEFXWRUY55NMZ2N74


#### Step 6: Bob proposes a transaction to the smart signature
* Again he proposes a payment from the cashmachine to **himself**
* The payment transaction is signed by the smart signature, **if the conditions are fullfilled** (correct password)

In [42]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0.1*1e6)
my_note        = '{"4711"}'             # correct password
note           = my_note.encode()

txn = PaymentTxn(sender=Cashmachine['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt,
                 note=note)

# Step 2: sign TX <---- This step is different!
encodedProg = Cashmachine['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19804148.
Waiting for round 19804148 to finish.
Waiting for round 19804149 to finish.
Transaction YYHKZ6UDBCTEPAIIFWCYKDNDMA7OQB3WEQP66OFLQJC53AIABGQA confirmed in round 19804150.


In [19]:
# Look on Algoexplorer.io at the smart signature. 
# - The password is visible in note/ASCII
print('http://testnet.algoexplorer.io/address/'+Cashmachine['hash'])

http://testnet.algoexplorer.io/address/N4HSKVGGAOGLGO66MID5GF5LDMBVTADDE6LXW5ABCIYQQEOPHU5ZWTXMVM


#### Charlie tries a withdrawl, but he uses the wrong password

In [43]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0.1*1e6)
my_note        = '{"4712"}'             # <-- WRONG password
note           = my_note.encode()

txn = PaymentTxn(sender=Cashmachine['hash'], sp=sp, 
                 receiver=Charlie['public'], amt=withdrawal_amt,
                 note=note)

# Step 2: sign TX <---- This step is different!
encodedProg = Cashmachine['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)               # print entire error message

# Step 4: wait for confirmation
# No need to wait for confirmation
#txinfo = wait_for_confirmation(algod_client, txid)

transaction {_struct:{} Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Lsig:{_struct:{} Logic:[3 32 1 255 132 210 222 7 38 1 8 123 34 52 55 49 49 34 125 49 5 40 18 34 34 18 16] Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Args:[]} Txn:{_struct:{} Type:pay Header:{_struct:{} Sender:GWCINQHW75IVFL53I65WUENTFY6XDS5NVN35UC4EXWEFXWRUY55NMZ2N74 Fee:{Raw:1000} FirstValid:19804191 LastValid:19805191 Note:[123 34 52 55 49 50 34 125] GenesisID:testnet-v1.0 GenesisHash:JBR3KGFEWPEE5SAQ6IWU6EEBZMHXD4CZU6WCBXWGF57XBZIJHIRA Group:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA Lease:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] RekeyTo:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ} KeyregT

#### Dina does not even know that there is a password

In [44]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0.1*1e6)

txn = PaymentTxn(sender=Cashmachine['hash'], sp=sp, 
                 receiver=Dina['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Cashmachine['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)               # print entire error message

# Step 4: wait for confirmation
# No need to wait for confirmation
#txinfo = wait_for_confirmation(algod_client, txid)

transaction {_struct:{} Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Lsig:{_struct:{} Logic:[3 32 1 255 132 210 222 7 38 1 8 123 34 52 55 49 49 34 125 49 5 40 18 34 34 18 16] Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Args:[]} Txn:{_struct:{} Type:pay Header:{_struct:{} Sender:GWCINQHW75IVFL53I65WUENTFY6XDS5NVN35UC4EXWEFXWRUY55NMZ2N74 Fee:{Raw:1000} FirstValid:19804208 LastValid:19805208 Note:[] GenesisID:testnet-v1.0 GenesisHash:JBR3KGFEWPEE5SAQ6IWU6EEBZMHXD4CZU6WCBXWGF57XBZIJHIRA Group:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA Lease:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] RekeyTo:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ} KeyregTxnFields:{_struct:{} Vote